# CS 20 : TensorFlow for Deep Learning Research
## Lecture 04 : Eager execution
### Custon training subclassing
Classifying mnist by using Tensorflow's eager execution.

This guide uses these high-level TensorFlow concepts:

* Enable an [eager execution](https://www.tensorflow.org/guide/eager?hl=ko) development environment,
* Import data with the [Datasets API](https://www.tensorflow.org/guide/datasets?hl=ko)
* Build model class by inheriting `tf.keras.Model` with TensorFlow's [Keras API](https://keras.io/getting-started/sequential-model-guide/) and  
  
  
* Reference
    + https://www.tensorflow.org/tutorials/eager/custom_training_walkthrough?hl=ko

### Setup

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
tf.enable_eager_execution()

1.12.0


### Define `SoftmaxClassifier` class and `loss_fn` function

In [2]:
class SoftmaxClassifier(keras.Model):
    def __init__(self, num_classes):
        super(SoftmaxClassifier, self).__init__()
        self.__dense = tf.keras.layers.Dense(units = num_classes)
        
    def call(self, inputs):
        score = self.__dense(inputs)
        return score

def loss_fn(model, x, y):
    ce_loss = tf.losses.sparse_softmax_cross_entropy(labels = y, logits = model(x))
    return ce_loss

#### Instantiate `model` instance

In [4]:
model = SoftmaxClassifier(num_classes = 10)

### Import dataset and split dataset into train, validation, test

In [5]:
(x_train, y_train), (x_tst, y_tst) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype(np.float32).reshape(-1, 784)
x_tst = x_tst.astype(np.float32).reshape(-1, 784)

In [6]:
tr_indices = np.random.choice(range(x_train.shape[0]), size = 55000, replace = False)

x_tr = x_train[tr_indices]
y_tr = y_train[tr_indices].astype(np.int32)

x_val = np.delete(arr = x_train, obj = tr_indices, axis = 0)
y_val = np.delete(arr = y_train, obj = tr_indices, axis = 0).astype(np.int32)

print(x_tr.shape, y_tr.shape)
print(x_val.shape, y_val.shape)

(55000, 784) (55000,)
(5000, 784) (5000,)


### Train the model

In [7]:
epochs = 10
batch_size = 32
learning_rate = .03

tr_dataset = tf.data.Dataset.from_tensor_slices((x_tr, y_tr))
tr_dataset = tr_dataset.shuffle(buffer_size = 10000)
tr_dataset = tr_dataset.batch(batch_size = batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size = batch_size)

opt = tf.train.AdamOptimizer(learning_rate = learning_rate)

In [8]:
for epoch in range(epochs):
    avg_tr_loss = 0
    avg_val_loss = 0
    tr_step = 0
    val_step = 0
    
    # train
    for x_mb, y_mb in tr_dataset:
        with tf.GradientTape() as tape:
            tr_loss = loss_fn(model = model, x = x_mb, y = y_mb)
        grads = tape.gradient(target = tr_loss, sources = model.variables)
        opt.apply_gradients(zip(grads, model.variables))
        
        avg_tr_loss += tr_loss
        tr_step += 1
    else:
        avg_tr_loss /= tr_step
        
    # validation
    for x_mb, y_mb in val_dataset:
        val_loss = loss_fn(model = model, x = x_mb, y = y_mb)
        avg_val_loss += val_loss
        val_step += 1
    else:
        avg_val_loss /= val_step
        
    print('epochs : {:2}, tr_loss : {:.3f}, val_loss : {:.3f}'.format(epoch + 1, avg_tr_loss, avg_val_loss))

epochs :  1, tr_loss : 130.425, val_loss : 126.328
epochs :  2, tr_loss : 149.994, val_loss : 166.825


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/kbs/.pyenv/versions/3.6.6/envs/tensorflow/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3265, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-4118c76d23dc>", line 11, in <module>
    grads = tape.gradient(target = tr_loss, sources = model.variables)
  File "/home/kbs/.pyenv/versions/3.6.6/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/eager/backprop.py", line 901, in gradient
    output_gradients=output_gradients)
  File "/home/kbs/.pyenv/versions/3.6.6/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/eager/imperative_grad.py", line 64, in imperative_grad
    output_gradients)
  File "/home/kbs/.pyenv/versions/3.6.6/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/eager/backprop.py", line 117, in _gradient_function
    return grad_fn(mock_op, *out_grads)
  File "/home/kbs/.pyenv/versions/3.6.6/envs/tensorflow/lib/python3.6/site-packa

KeyboardInterrupt: 